# Building a Spam Filter using Naive Bayes

## Introduction

In this project we are going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm.

To classify messages as spam or non-spam, the computer:
* Learns how humans classify messages.
* Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
* Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where we can also find some of the authors' papers.

## Exploring the dataset

We'll start exploring the dataset:

In [1]:
import pandas as pd
sms_spam = pd.read_csv("SMSSpamCollection", sep="\t", header = None, names = ["Label", "SMS"])

In [2]:
print("The number of rows is: ",sms_spam.shape[0])
print("The number of columns is: ",sms_spam.shape[1])


The number of rows is:  5572
The number of columns is:  2


In [3]:
sms_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


We'll find what percentage of the messages is spam and what percentage is ham (ham means non-spam)

In [4]:
ham_percentage = ((sms_spam["Label"].value_counts()[0])/sms_spam.shape[0]) * 100
print("The ham percentage is: {:.2f}%".format(ham_percentage))

The ham percentage is: 86.59%


In [5]:
spam_percentage = ((sms_spam["Label"].value_counts()[1])/sms_spam.shape[0]) * 100
print("The spam percentage is: {:.2f}%".format(spam_percentage))

The spam percentage is: 13.41%


## Training and test set

Now that we've become a bit familiar with the dataset, we can move on to building the spam filter. 
However, before creating it, it's very helpful to first think of a way of testing how well it works. 
We are going to keep 80% of our dataset for training, and 20% for testing. The dataset has 5,572 messages, which means that:
* The training set will have 4,458 messages (about 80% of the dataset), which we'll use to "train" the computer how to classify messages.
* The test set will have 1,114 messages (about 20% of the dataset), which we'll use to test how good the spam filter is with classifying new messages.


For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

We'll start randomizing the entire dataset by using the DataFram.sample() method and we'll use random_state = 1 to make sure our results are reproducible:

In [6]:
random_sms_spam = sms_spam.sample(frac = 1, random_state = 1)

We'll split the randomized dataset into a training and a test set:

In [7]:
training_set = random_sms_spam.iloc[0:4458].copy()
training_set = training_set.reset_index(drop = True)
training_set_ham = ((training_set["Label"].value_counts()[0])/training_set.shape[0]) * 100
print("The ham percentage in the training set is: {:.2f}%".format(training_set_ham))
training_set_spam = ((training_set["Label"].value_counts()[1])/training_set.shape[0]) * 100
print("The spam percentage in the training set is: {:.2f}%".format(training_set_spam))

The ham percentage in the training set is: 86.54%
The spam percentage in the training set is: 13.46%


In [8]:
test_set = random_sms_spam.iloc[4458:].copy()
test_set = test_set.reset_index(drop = True)

test_set_ham = ((test_set["Label"].value_counts()[0])/test_set.shape[0]) * 100
print("The ham percentage in the training set is: {:.2f}%".format(test_set_ham))

test_set_spam = ((test_set["Label"].value_counts()[1])/test_set.shape[0]) * 100
print("The spam percentage in the training set is: {:.2f}%".format(test_set_spam))

The ham percentage in the training set is: 86.80%
The spam percentage in the training set is: 13.20%


As we can see, the ham and spam percentages are similar in both sets.

## Letter case and punctuation

Previously, we split our dataset into a training set and a test set. The next big step is to use the training set to teach the algorithm to classify new messages.
To calculate the probabilities we'll use in our algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will aloow us to extract easily all the information we need.


We are going to remove all the punctuation from the SMS column:

In [9]:
import re
def only_characters(text):
    clean = re.sub('\W', ' ', text )
    return clean

training_set["SMS"] = training_set["SMS"].apply(only_characters)
training_set.head()

,Label,SMS
0,ham,Yep by the pretty sculpture
1,ham,Yes princess Are you going to make me moan
2,ham,Welp apparently he retired
3,ham,Havent
4,ham,I forgot 2 ask ü all smth There s a card on ...


And now we are going to transform every letter in every word to lower case:

In [10]:
training_set["SMS"] = training_set["SMS"].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Creating the vocabulary

Now we want to create a list with all the vocabulary in the messages of the training set. The vocabulary is the set of unique words in the messages.

We'll begin by transforming each message from the SMS column into a list by splitting the string at the space character:

In [11]:
training_set["SMS"] = training_set["SMS"].str.split()
training_set.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


And then we'll create the vocabulary list with the words of the training_set:

In [12]:
vocabulary = []
for sms in training_set["SMS"]:
    for s in sms:
        vocabulary.append(s)
        
#Remove th duplicates
vocabulary = set(vocabulary)

#Transform into a list again
vocabulary = list(vocabulary)
print(len(vocabulary))

7783


## The final Training set

In this step we want to make this data transformation:

From this:

| |Label | SMS|
|------|------|------|
|   0  | spam| SECRET PRIZE! CLAIM SECRET PRIZE NOW!!|
|1|ham|	Comint to my secret party?|
|2|	spam|Winner! Claim secret prize now!|

To this:

| |Label | secret |prize	|claim|	now|	coming|	to|	my|	party|	winner|
|------|------|------|------|------|------|------|------|------|------|------|
|   0  | spam|2|	2|	1|	1|	0|	0|	0|	0|	0|
|1|ham|	1|	0|	0|	0|	1|	1|	1|	1|	0|
|2|	spam|1|	1|	1|	1|	0|	0|	0|	0|	1|

Eventually, we're going to create a new DataFrame. However, we'll first build a dictionary that we'll then convert to the DataFrame we need.


In [13]:
#The code [0] * len(training_set["SMS]) outputs a list of the lenght of training_set["SMS"], where each element in the list will be a 0
word_counts_per_sms = {unique_word: [0] * len(training_set["SMS"]) for unique_word in vocabulary}

#We loop over training_set['SMS'] using at the same time the enumerate() function to get both the index and the SMS message (index and sms).
for index, sms in enumerate(training_set["SMS"]):
    for word in sms:
        word_counts_per_sms[word][index] +=1
    

Now, using the dictionary that we just created, we'll transform it into a DataFrame:

In [14]:
word_counts_sms = pd.DataFrame(word_counts_per_sms)
word_counts_sms.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


Now we'll concatenate this DataFrame with the training_set (this way, we'll also have the Label and the SMS columns):

In [15]:
training_set_clean = pd.concat([training_set, word_counts_sms], axis = 1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating constants first

Now that we're done with data cleaning and we have a training set to work with, we can begin creating the spam filter using the Naive Bayes algoithm.

To decide if a message is spam or not, the algortihm needs to answer two probability questions:

<img src="images/Spam filter using Naive Bayes 02.jpg" width="550px" />

And also, to calculate P(wi|Spam) and P(wi|Spam) inside the formulas above, we'll nedd to use these equations:

<img src="images/Spam filter using Naive Bayes 02.jpg"/>

Some of the terms in the four equations will have the same value for every message, then we'll calculate these values once to avoid doing the computations again when a new message comes in.

In [16]:
total_m = training_set_clean["Label"].value_counts(dropna = False)
total_m

ham     3858
spam     600
Name: Label, dtype: int64

In [17]:
p_spam = total_m[1]/(total_m[0] + total_m[1])
p_spam 

0.13458950201884254

In [18]:
p_ham = total_m[0]/(total_m[0] + total_m[1])
p_ham

0.8654104979811574

In [19]:
training_ham = training_set_clean[training_set_clean["Label"]=="ham"]
training_spam = training_set_clean[training_set_clean["Label"]=="spam"]

n_spam = training_spam["SMS"].apply(len).sum()
n_ham = training_ham["SMS"].apply(len).sum()
n_vocabulary = len(vocabulary)
print("Number of words in all the spam messages: ",n_spam)
print("Number of words in all the non-spam messages: ",n_ham)
print("Number of words in vocabulary: ",n_vocabulary)

Number of words in all the spam messages:  15190
Number of words in all the non-spam messages:  57237
Number of words in vocabulary:  7783


We'll also use Laplace smoothing and set alpha = 1:

In [20]:
alpha = 1

## Calculating parameters

Now we'll calculate the parameters of the algorithm. The parameters are the probability of every word in the vocabulary given spam and given ham.

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(wi|Spam) and P(wi|Ham).

The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

As we said before, if we didn't calculate all these values beforehand, then all these calculations would need to be done every time a new message comes in.

In [21]:
parameter_spam = {i: 0 for i in vocabulary}
parameter_ham = {i: 0 for i in vocabulary}

In [22]:
#Calculate parameters
for w in vocabulary:
    #Number of w in the spam messages
    n_w_given_spam = training_spam[w].sum()
    #Probability of the word given spam
    p_w_given_spam = (n_w_given_spam + alpha)/(n_spam + alpha*n_vocabulary)
    #Add to the dictionary the parameter
    parameter_spam[w] = p_w_given_spam
    
    n_w_given_ham = training_ham[w].sum()
    p_w_given_ham = (n_w_given_ham + alpha)/(n_ham + alpha * n_vocabulary)
    parameter_ham[w] = p_w_given_ham


## Classifying a new message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:
* Takes in as input a new message (w1, w2, ..., wn)
* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
* Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
* If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
* If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
* If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [23]:
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()


    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for w in message:
        if w in parameter_spam:
            p_spam_given_message *= parameter_spam[w]
        if w in parameter_ham:
            p_ham_given_message *= parameter_ham[w]        
   

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [24]:
classify("WINNER!! This is the secret code to unlock the money: C3421.")

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [25]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the spam filter's accuracy

Previously we created the spam filter and we classified two new messages. We'll now try to determinate how well the spam filter does  on our test_setof 1,114 messages.

First off, we'll change the classify() function that we wrote previously to return the labels instead of printing them:

In [26]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()


    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for w in message:
        if w in parameter_spam:
            p_spam_given_message *= parameter_spam[w]
        if w in parameter_ham:
            p_ham_given_message *= parameter_ham[w]        

    if p_ham_given_message > p_spam_given_message:
        return"ham"
    elif p_ham_given_message < p_spam_given_message:
        return"spam"
    else:
        return"needs human classification"

Now that we have a function that returns labels insted of printing them, we can use it to create a new column in our test set:

In [27]:
test_set["predicted"] = test_set["SMS"].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we'll measure the accuracy of our spam filter to find out how well our spam filter does.

In [28]:
correct = 0
total = test_set.shape[0]

In [29]:
for index, row in test_set.iterrows():
    if row["Label"] == row["predicted"]:
        correct += 1
print(correct)

1100


In [30]:
accuracy = (correct/total) * 100
print("Correct: ", correct)
print("Incorrect: ", total - correct)
print("The accuracy fo the filter is: {:.2f}%".format(accuracy))

Correct:  1100
Incorrect:  14
The accuracy fo the filter is: 98.74%
